In [43]:
import pandas as pd
import re

In [44]:
def to_scientific_notation(value, decimal_places=4):
    """
    숫자를 지정된 소수점 자릿수의 지수 표기법으로 변환합니다.
    """
    format_string = f'{{:.{decimal_places}e}}'
    return format_string.format(value)

def convert_column_to_scientific(df, column_name, decimal_places=4):
    """
    DataFrame의 특정 열을 지수 표기법으로 변환합니다.
    """
    df[column_name] = df[column_name].apply(lambda x: to_scientific_notation(x, decimal_places))
    return df

In [45]:
# general information
file_path = './kinetic_DB/'
i_list = list(range(1,250))
#i_list = [101]

In [46]:
# bolsigdb.dat 생성
with open('bolsigdb.dat','w') as file:
    for i in i_list:
        file_name = 'reaction_%d.csv'%i
        f = pd.read_csv(file_path+file_name)
        f = convert_column_to_scientific(f, 'Energy(eV)')
        f = convert_column_to_scientific(f, 'Cross_section(m2)')

        type = f['Type'].loc[0]
        reaction = f['Reaction'].loc[0]
        if type == 'ELASTIC' or 'EFFECTIVE':
            eloss = str(f['ELoss'].loc[0])+' / mass ratio'
        else:
            eloss = str(f['ELoss'].loc[0])+' / threshold energy'

        line = '-----------------------------'

        file.write(type+'\n')
        file.write(reaction+'\n')
        file.write(eloss+'\n')
        file.write(line+'\n')
        for j in range(len(f)):
            file.write('  '+str(f['Energy(eV)'].loc[j])+'  '+str(f['Cross_section(m2)'].loc[j])+'\n')
        file.write(line+'\n')
        file.write('\n')
    
    file.close()

In [47]:
# kinep.inp 생성
with open('kinet.inp','w') as file:
    species = []
    for i in i_list:
        file_name = 'reaction_%d.csv'%i
        f = pd.read_csv(file_path+file_name)
        species += f['Formula'].loc[0].split(' ')
    
    f2 = pd.read_csv(file_path+"reaction_gas.csv")
    for i in range(len(f2)):
        species += f2['Reaction'].iloc[i].split(' ')
    species = list(set(species))
    species.remove('+')
    species.remove('->')
    species.remove('=>')
    species.remove('E')
    species.remove('e')
    species = ['e']+species

    if len(species) > 31:
        species1 = species[:31]
        species2 = species[31:]
    else:
        species1 = species
        species2 = []

    species1 = ' '.join(species1)
    species2 = ' '.join(species2)    

    file.write('ELEMENTS'+'\n')
    file.write('e C H'+'\n')
    file.write('END'+'\n')
    file.write('\n')
    file.write('SPECIES'+'\n')
    file.write(species1+'\n')
    if species2 != []:
        file.write(species2+'\n')
    file.write('END'+'\n')
    file.write('\n')

    bolsig = []
    for i in i_list:
        file_name = 'reaction_%d.csv'%i
        f = pd.read_csv(file_path+file_name)
        bolsig += f['Species'].loc[0].split(' ')
    bolsig = list(set(bolsig))
    bolsig.remove('/')
    bolsig.remove('e')
    bolsig = ' '.join(bolsig)
    bolsig = bolsig
    
    file.write('BOLSIG'+'\n')
    file.write(bolsig+'\n')
    file.write('END'+'\n')
    file.write('\n')

    file.write('REACTIONS'+'\n')

    file.write('#'+'\n')
    file.write('# Kinetic parameters'+'\n')
    file.write('#'+'\n')
    file.write('$ double precision, parameter :: R = 8.314d-3'+'\n')
    file.write('$ double precision, parameter :: f0 = 1.000d0'+'\n')
    for i in range(38):
        para = 'f' + str(i+1)
        file.write('$ double precision, parameter :: ' + para + '= 1.000d0'+'\n')

    file.write('#'+'\n')
    file.write('# electron collision reaction'+'\n')
    file.write('#'+'\n')

    for i in i_list:
        file_name = 'reaction_%d.csv'%i
        f = pd.read_csv(file_path+file_name)
        if f['Type'].loc[0] != 'ELASTIC' and f['Type'].loc[0] != 'EFFECTIVE':
            formula = f['Formula'].loc[0]
            formula = formula.replace('E','e').replace('->','=>')
            formula = formula
            bolsig_comp = formula.replace('e','').replace('+','').replace('=>','->')
            bolsig_comp = bolsig_comp.split(' ')
            bolsig_comp = [item for item in bolsig_comp if item != '']
            bolsig_comp = ' '.join(bolsig_comp)
            if f['Type'].loc[0] == 'IONIZATION':
                bolsig_comp += '+'
            bolsig_comp = bolsig_comp
            file.write(formula)
            file.write('\t'*3)
            file.write('! f0 * Bolsig+ '+ bolsig_comp)
            file.write('\n')
    file.write('#'+'\n')

    file.write('# Ion recombination reaction'+'\n')
    file.write('#'+'\n')
    file.write('e + e + CH4^+ => e + CH4')
    file.write('\t'*3)
    file.write('! f0')
    file.write('\n')
    file.write('e + e + CH3^+ => e + CH3')
    file.write('\t'*3)
    file.write('! f0')
    file.write('\n')
    file.write('e + e + CH2^+ => e + CH2')
    file.write('\t'*3)
    file.write('! f0')
    file.write('\n')
    file.write('e + e + CH^+ => e + CH')
    file.write('\t'*3)
    file.write('! f0')
    file.write('\n')
    file.write('e + e + C2H6^+ => e + C2H6')
    file.write('\t'*3)
    file.write('! f0')
    file.write('\n')
    file.write('e + e + C2H5^+ => e + C2H5')
    file.write('\t'*3)
    file.write('! f0')
    file.write('\n')
    file.write('e + e + C2H4^+ => e + C2H4')
    file.write('\t'*3)
    file.write('! f0')
    file.write('\n')
    file.write('e + e + C2H3^+ => e + C2H3')
    file.write('\t'*3)
    file.write('! f0')
    file.write('\n')
    file.write('e + e + C2H2^+ => e + C2H2')
    file.write('\t'*3)
    file.write('! f0')
    file.write('\n')
    file.write('e + e + C3H8^+ => e + C3H8')
    file.write('\t'*3)
    file.write('! f0')
    file.write('\n')
    file.write('e + e + C3H7^+ => e + C3H7')
    file.write('\t'*3)
    file.write('! f0')
    file.write('\n')
    file.write('e + e + C3H6^+ => e + C3H6')
    file.write('\t'*3)
    file.write('! f0')
    file.write('\n')
    file.write('e + e + C3H5^+ => e + C3H5')
    file.write('\t'*3)
    file.write('! f0')
    file.write('\n')
    file.write('e + e + C3H4^+ => e + C3H4')
    file.write('\t'*3)
    file.write('! f0')
    file.write('\n')
    file.write('e + e + H^+ => e + H')
    file.write('\t'*3)
    file.write('! f0')
    file.write('\n')
    file.write('e + e + H2^+ => e + H2')
    file.write('\t'*3)
    file.write('! f0')
    file.write('\n')
    file.write('#'+'\n')
    file.write('# Ion dissociative reaction'+'\n')
    file.write('#'+'\n')
    for i in range(21):
        file.write(f2['Reaction'].iloc[i])
        file.write('\t'*3)
        file.write('! f0 * ' + f2['Value'].iloc[i])
        file.write('\n')
    file.write('#'+'\n')
    file.write('# Ion-Neutral reaction'+'\n')
    file.write('#'+'\n')
    for i in range(97):
        file.write(f2['Reaction'].iloc[i+21])
        file.write('\t'*3)
        file.write('! f0 * ' + f2['Value'].iloc[i+21])
        file.write('\n')
    file.write('#'+'\n')
    file.write('# Neutral-Neutral reaction'+'\n')
    file.write('#'+'\n')
    para =  ['f1 * f3 * f21 * f21 * ', 'f1 * f4 * f26 * f37 * ', 'f1 * f6 * f24 * f21 * ', 'f1 * f8 * f26 * f21 * ', 'f1 * f10 * f28 * f21 * ', 'f1 * f12 * f30 * f21 * ', 'f1 * f14 * f32 * f21 * ', 'f1 * f18 * f21 * f38 * ', 'f1 * f2 * f37 * f24 * ', 'f1 * f16 * f34 * f21 * ', 'f1 * f3 * f24 * ', 'f1 * f21 * f37 * ', 'f2 * f2 * f25 * f37 * ', 'f2 * f2 * f24 * ', 'f2 * f3 * f26 * f37 * ', 'f2 * f5 * f25 * f20 * ', 'f2 * f6 * f30 * ', 'f2 * f7 * f27 * f20 * ', 'f2 * f8 * f28 * f20 * ', 'f2 * f8 * f32 * ', 'f2 * f9 * f29 * f20 * ', 'f2 * f11 * f31 * f20 * ', 'f2 * f12 * f32 * f20 * ', 'f2 * f13 * f33 * f20 * ', 'f2 * f19 * f20 * f37 * ', 'f2 * f18 * f22 * f38 * ', 'f2 * f18 * f20 * ', 'f2 * f12 * f25 * f25 * ', 'f2 * f22 * f37 * ', 'f2 * f23 * f38 * ', 'f2 * f6 * f24 * f22 * ', 'f3 * f3 * f28 * f37 * f37 * ', 'f3 * f6 * f26 * f21 * ', 'f3 * f8 * f28 * f21 * ', 'f3 * f10 * f28 * f23 * ', 'f3 * f11 * f31 * f21 * ', 'f3 * f12 * f26 * f25 * ', 'f3 * f12 * f32 * f21 * ', 'f3 * f13 * f33 * f21 * ', 'f3 * f19 * f21 * f37 * ', 'f3 * f18 * f23 * f38 * ', 'f3 * f23 * f37 * ', 'f3 * f3 * f28 * f38 * ', 'f3 * f18 * f21 * ', 'f4 * f5 * f32 * f37 * ', 'f4 * f5 * f31 * ', 'f4 * f19 * f22 * f37 * ', 'f4 * f2 * f27 * f37 * ', 'f4 * f3 * f28 * f37 * ', 'f4 * f19 * f21 * ', 'f4 * f8 * f22 * f28 * ', 'f5 * f8 * f25 * f26 * ', 'f5 * f12 * f30 * f25 * ', 'f5 * f14 * f32 * f25 * ', 'f5 * f18 * f25 * f38 * ', 'f5 * f18 * f20 * f21 * ', 'f5 * f21 * f21 * ', 'f5 * f16 * f34 * f25 * ', 'f5 * f4 * f26 * f21 * ', 'f5 * f3 * f25 * f21 * ', 'f6 * f6 * f24 * f26 * ', 'f6 * f7 * f24 * f27 * ', 'f6 * f9 * f24 * f29 * ', 'f6 * f10 * f26 * f28 * ', 'f6 * f11 * f24 * f31 * ', 'f6 * f12 * f30 * f26 * ', 'f6 * f12 * f32 * f24 * ', 'f6 * f13 * f33 * f24 * ', 'f6 * f19 * f24 * f37 * ', 'f6 * f18 * f21 * f21 * ', 'f6 * f18 * f26 * f38 * ', 'f6 * f18 * f24 * ', 'f6 * f26 * f37 * ', 'f6 * f6 * f34 * ', 'f6 * f16 * f26 * f34 * ', 'f6 * f8 * f26 * f26 * ', 'f7 * f18 * f27 * f38 * ', 'f7 * f18 * f25 * ', 'f7 * f19 * f25 * f37 * ', 'f7 * f27 * f37 * ', 'f7 * f13 * f33 * f25 * ', 'f7 * f9 * f27 * f27 * ', 'f7 * f13 * f27 * f31 * ', 'f7 * f7 * f25 * f27 * ', 'f7 * f2 * f31 * ', 'f7 * f28 * f38 * ', 'f7 * f6 * f35 * ', 'f7 * f19 * f24 * ', 'f7 * f3 * f32 * ', 'f7 * f16 * f32 * f31 * ', 'f8 * f8 * f26 * f28 * ', 'f8 * f11 * f26 * f31 * ', 'f8 * f12 * f30 * f28 * ', 'f8 * f12 * f32 * f26 * ', 'f8 * f13 * f33 * f26 * ', 'f8 * f14 * f32 * f28 * ', 'f8 * f19 * f26 * f37 * ', 'f8 * f18 * f28 * f38 * ', 'f8 * f18 * f26 * ', 'f8 * f28 * f37 * ', 'f8 * f16 * f28 * f34 * ', 'f9 * f18 * f27 * ', 'f9 * f19 * f26 * ', 'f9 * f19 * f27 * f37 * ', 'f9 * f2 * f33 * ', 'f9 * f16 * f32 * f33 * ', 'f11 * f14 * f32 * f31 * ', 'f11 * f18 * f31 * f38 * ', 'f11 * f25 * f21 * ', 'f11 * f16 * f34 * f31 * ', 'f11 * f3 * f34 * ', 'f12 * f12 * f32 * f30 * ', 'f12 * f13 * f33 * f30 * ', 'f12 * f14 * f32 * f32 * ', 'f12 * f19 * f30 * f37 * ', 'f12 * f18 * f32 * f38 * ', 'f12 * f18 * f30 * ', 'f12 * f18 * f21 * f25 * ', 'f12 * f32 * f37 * ', 'f12 * f26 * f21 * ', 'f12 * f16 * f34 * f32 * ', 'f13 * f9 * f27 * f33 * ', 'f13 * f13 * f31 * f33 * ', 'f13 * f33 * f37 * ', 'f13 * f18 * f33 * f38 * ', 'f13 * f18 * f31 * ', 'f13 * f21 * f27 * ', 'f13 * f2 * f35 * ', 'f13 * f16 * f34 * f33 * ', 'f14 * f19 * f32 * f37 * ', 'f14 * f18 * f32 * ', 'f14 * f28 * f21 * ', 'f16 * f26 * f25 * ', 'f16 * f3 * f26 * f31 * ', 'f16 * f32 * f21 * ', 'f16 * f19 * f34 * f37 * ', 'f15 * f2 * f20 * f35 * ', 'f15 * f31 * f21 * ', 'f15 * f25 * f25 * ', 'f15 * f18 * f35 * f38 * ', 'f15 * f3 * f35 * f21 * ', 'f15 * f8 * f26 * f35 * ', 'f15 * f12 * f30 * f35 * ', 'f15 * f10 * f28 * f35 * ', 'f15 * f6 * f24 * f35 * ', 'f15 * f14 * f32 * f35 * ', 'f15 * f3 * f36 * ', 'f17 * f21 * f35 * ', 'f19 * f37 * f37 * ', 'f18 * f18 * f38 * ']
    index = 0
    for i in range(len(f2)-118):
        file.write(f2['Reaction'].iloc[i+118])
        file.write('\t'*3)
        file.write('! '+ para[index] + f2['Value'].iloc[i+118])
        index += 1
        file.write('\n')

    file.write('END')

In [48]:
f2 = pd.read_csv(file_path+"reaction_gas.csv")

In [49]:
f2

,Unnamed: 0,Reaction,Value,Number
0,0,e + CH5^+ => CH3 + H + H,2.57d-07 * (300./Tgas)**0.3,NaN
1,1,e + CH5^+ => CH2 + H2 + H,6.61d-08 * (300./Tgas)**0.3,NaN
2,2,e + CH4^+ => CH3 + H,1.18d-08 * (300./Tgas)**0.5,NaN
3,3,e + CH4^+ => CH2 + H + H,2.42d-08 * (300./Tgas)**0.5,NaN
4,4,e + CH4^+ => CH + H2 + H,1.41d-08 * (300./Tgas)**0.5,NaN
...,...,...,...,...
263,263,C4H9H + C3H5 => C3H6 + C4H9,8.63d-14*(Tgas/298.)**3.30*exp(-83.06/(R*Tgas)),57.0
264,264,C4H9H + CH2 => C5H12,9.61d-13,94.0
265,265,C5H12 => CH3 + C4H9,3.16d16*exp(-331./(R*Tgas)),105.0
266,266,H2 => H + H,1.88d-8*(Tgas/298.)**(-1.10)*exp(-437./(R*Tgas)),106.0
